Supply demand forecasting and capacity Optmization

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#LOAD DATA
df = pd.read_csv("azure_demand_uncleaned_10k.csv")

print("Initial Shape:", df.shape)
print(df.head())


Initial Shape: (10000, 14)
         date         region service_type  demand_units  capacity_allocated  \
0  2022-01-01         eastus      Compute        1772.0              2119.0   
1  2022-01-01         eastus      Storage         569.0               772.0   
2  2022-01-01     westeurope      Compute        2064.0              2493.0   
3  2022-01-01     westeurope      Storage         485.0               674.0   
4  2022-01-01  southeastasia      Compute         434.0               518.0   

   cost_usd  availability  is_weekend  market_demand_index  gdp_growth  \
0    154.83         99.93           1               107.29        1.53   
1     12.16        100.80           1                94.17        2.20   
2    168.26         99.94           1               104.12        4.44   
3     11.00         99.89           1                89.88        3.46   
4     37.83         99.91           1                95.40        3.07   

   customer_growth_rate  pricing_event  service_incid

In [ ]:

df["date"] = pd.to_datetime(df["date"], errors="coerce")

# Drop empty daterows
df = df.dropna(subset=["date"])


In [ ]:
#REMOVE DUPLICATES
df = df.drop_duplicates()


In [ ]:
# HANDLE MISSING VALUES

# Fill demand using region+service median
df["demand_units"] = df.groupby(
    ["region", "service_type"]
)["demand_units"].transform(
    lambda x: x.fillna(x.median())
)
# Fill capacity using 1.25 × demand if missing
df["capacity_allocated"] = df["capacity_allocated"].fillna(
    df["demand_units"] * 1.25
)

# cost using demand × price logic
compute_price = 0.085
storage_price = 0.022

def fill_cost(row):
    if pd.isna(row["cost_usd"]):
        if row["service_type"] == "Compute":
            return row["demand_units"] * compute_price
        else:
            return row["demand_units"] * storage_price
    return row["cost_usd"]

df["cost_usd"] = df.apply(fill_cost, axis=1)


In [ ]:
# FIX INVALID AVAILABILITY

# Remove unrealistic SLA (>100%)
df.loc[df["availability"] > 100, "availability"] = 99.99

# Clip to realistic SLA range
df["availability"] = df["availability"].clip(99.0, 100.0)

In [ ]:
#HANDLE OUTLIERS (IQR METHOD)

def remove_outliers(column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return df[column].clip(lower, upper)

df["demand_units"] = remove_outliers("demand_units")
df["capacity_allocated"] = remove_outliers("capacity_allocated")



In [ ]:

# FIX DATA TYPES

numeric_columns = [
    "demand_units",
    "capacity_allocated",
    "cost_usd",
    "availability",
    "market_demand_index",
    "gdp_growth",
    "customer_growth_rate",
    "industry_mix_index"
]
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors="coerce")

In [ ]:
# SORT DATA
df = df.sort_values(by=["region", "service_type", "date"])

In [ ]:
print("\nAfter Cleaning Shape:", df.shape)
print("\nMissing Values After Cleaning:\n", df.isnull().sum())

# SAVE CLEAN DATA

df.to_csv("azure_demand_cleaned.csv", index=False)

print("\n✅ Cleaned dataset saved as 'azure_demand_cleaned.csv'")


After Cleaning Shape: (10000, 14)

Missing Values After Cleaning:
 date                    0
region                  0
service_type            0
demand_units            0
capacity_allocated      0
cost_usd                0
availability            0
is_weekend              0
market_demand_index     0
gdp_growth              0
customer_growth_rate    0
pricing_event           0
service_incident        0
industry_mix_index      0
dtype: int64

✅ Cleaned dataset saved as 'azure_demand_cleaned.csv'


Recheck

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("azure_demand_cleaned.csv")

df["date"] = pd.to_datetime(df["date"])

print("Shape:", df.shape)


print("\n🔎 Missing Values Check:")
print(df.isnull().sum())



print("\n🔎 Time Continuity Check:")

time_issues = []

for (region, service), group in df.groupby(["region", "service_type"]):
    group = group.sort_values("date")
    date_diff = group["date"].diff().dropna()

    # Check if gaps > 1 day exist
    if (date_diff > pd.Timedelta(days=1)).any():
        time_issues.append((region, service))

if len(time_issues) == 0:
    print("✅ No time gaps found.")
else:
    print("⚠ Time gaps found in:")
    for issue in time_issues:
        print(issue)


print("\n🔎 Demand Distribution:")
print(df["demand_units"].describe())

print("\n🔎 Capacity Distribution:")
print(df["capacity_allocated"].describe())


Shape: (10000, 14)

🔎 Missing Values Check:
date                    0
region                  0
service_type            0
demand_units            0
capacity_allocated      0
cost_usd                0
availability            0
is_weekend              0
market_demand_index     0
gdp_growth              0
customer_growth_rate    0
pricing_event           0
service_incident        0
industry_mix_index      0
dtype: int64

🔎 Time Continuity Check:
✅ No time gaps found.

🔎 Demand Distribution:
count    10000.000000
mean       904.940000
std        527.112129
min         97.000000
25%        509.000000
50%        786.000000
75%       1201.000000
max       2239.000000
Name: demand_units, dtype: float64

🔎 Capacity Distribution:
count    10000.000000
mean      1157.436197
std        694.649363
min        120.000000
25%        626.187500
50%        996.000000
75%       1535.000000
max       2898.218750
Name: capacity_allocated, dtype: float64


In [ ]:
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month
df["day"] = df["date"].dt.day
df["day_of_week"] = df["date"].dt.dayofweek
df["quarter"] = df["date"].dt.quarter

In [ ]:
import pandas as pd

# Reload the cleaned data to ensure 'region' and 'service_type' columns are present
df = pd.read_csv("azure_demand_cleaned.csv")
df["date"] = pd.to_datetime(df["date"])

# Re-create date-related features
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month
df["day"] = df["date"].dt.day
df["day_of_week"] = df["date"].dt.dayofweek
df["quarter"] = df["date"].dt.quarter

print("DataFrame reloaded and date features recreated.")

DataFrame reloaded and date features recreated.


In [ ]:
df["lag_1"] = df.groupby(
    ["region", "service_type"]
)["demand_units"].shift(1)

df["lag_7"] = df.groupby(
    ["region", "service_type"]
)["demand_units"].shift(7)

df["rolling_mean_7"] = df.groupby(
    ["region", "service_type"]
)["demand_units"].transform(lambda x: x.rolling(7).mean())


In [ ]:
split_date = df["date"].quantile(0.8)

train = df[df["date"] <= split_date]
test = df[df["date"] > split_date]

In [ ]:
df = df.dropna(subset=["lag_1", "lag_7", "rolling_mean_7"])

In [ ]:
print(df.head())

         date     region service_type  demand_units  capacity_allocated  \
7  2022-01-08  centralus      Compute        1170.0              1462.5   
8  2022-01-09  centralus      Compute        1629.0              1962.0   
9  2022-01-10  centralus      Compute        1221.0              1532.0   
10 2022-01-11  centralus      Compute         417.0               554.0   
11 2022-01-12  centralus      Compute        1972.0              2535.0   

    cost_usd  availability  is_weekend  market_demand_index  gdp_growth  ...  \
7      96.33         99.97           1               101.61        2.23  ...   
8     145.04         99.96           1                90.71        3.07  ...   
9     106.85         99.95           0                87.74        2.70  ...   
10     35.43         99.86           0                96.01        3.45  ...   
11    161.91         99.88           0                89.56        1.71  ...   

    service_incident  industry_mix_index  year  month  day  day_of_w